# Projeto de Recuperação da Informação

## Sistema de recomendação de filmes usando banco com notas de usuários  

O projeto consistiu em usar uma base de dados do [MovieLens](https://grouplens.org/datasets/movielens/100k/) que tinha avaliações de usuários para filmes assistidos. Baseado nesses dados e usando uma biblioteca de recomendação ([SurpriseLib](http://surpriselib.com/)) foram recomendados filmes para novos usuários.

In [27]:
import numpy
import pandas as pd
import os

## 1. Os dados 

O MovieLens fornece uma base rica com muitas tabelas diferentes com informações sobre filmes, dados de usuários, gêneros de filmes, etc. Para esse estudo, no entanto, foram usadas duas tabelas: a que possuía informações sobre os filmes (u.item) e a que possuia as notas de avaliação* (u.data). Um sample dos dados segue:

__u.data:__

In [4]:
file = open("../back/ml-100k/u_backup.data", "r", encoding='ISO-8859-1')
lines = file.readlines()
sample_ratings = []
for line in lines[:5]:
        columns = line.split('\t')
        sample_ratings.append({"user_id":  columns[0], "movie_id":  columns[1], "rating":  columns[2]})

df = pd.DataFrame(data=sample_ratings)
cols = ["user_id", "movie_id", "rating"]
df = df[cols]
df

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


_*As notas foram avaliadas entre 0 e 5_

__u.item:__

In [39]:
file = open("../back/ml-100k/u.item", "r", encoding='ISO-8859-1')
lines = file.readlines()
movies = []
for line in lines:
    columns = line.split('|')
    movie_id = columns[0]
    name = columns[1]
    movies.append({"movie_id": movie_id, "name": name})href
        
df = pd.DataFrame(data=movies[:5])
cols = ["movie_id", "name"]
df = df[cols]
df

,movie_id,name
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


## 2. O algoritmo

### 2.1 Injeção de dados

A ideia do algoritmo é modificar os dados recebidos como input, adicionando novas linhas com filmes que o usuário gostou. Para tal, precisa-se de uma função que possa escrever uma lista de ids de filmes na tabela de notas:

In [24]:
DEFAULT_USER = "1234"
MAX_RATING = "5"
DEFAULT_TIMESTAMP = "000000000"

def add_movie_list(movies):
    """
    Adiciona filmes para a tabela de notas MovieLens
    :param movies: array de id de filmes que o usuário já assistiu e gostou
    :return:
    """
    
    f = open("../back/ml-100k/u.data", "a")

    for movie in movies:
        f.write("%s\t%s\t%s\t%s\n" % (DEFAULT_USER, movie, MAX_RATING, DEFAULT_TIMESTAMP))

    f.close()

__Note que:__ 
    1. Tem-se um id de usuário inventado e padrão, que serve somente para que possa ser possível a inserção de novos dados na tabela; 
    2. Existe também um timestamp padrão, pois não há interesse em estudar a data de avaliação
    3. As inserções são sempre com nota máxima, pois deseja-se facilitar a experiência do usuário e leva-se em conta que ele adicionará a sua lista, filmes que gostou muito

### 2.2 Treinamento

Usando o método que foi criado, e em posse do arquivo modificado, precisa-se agora treinar o modelo para que ele possa __prever as notas que o novo usuário daria para os outros filmes__, para isso, usa-se a biblioteca __Surprise__.

In [29]:
from surprise import KNNBasic 
from surprise import Reader
from surprise import Dataset

def train(movie_list):
    """
    Treina um modelo customizado com os dados que o usuário forneceu
    :param movies: array de id de filmes que o usuário já assistiu e gostou
    :return:
    """
    add_movie_list(movie_list)

    #Carregando dados customizados no Surprise
    file_path = os.path.expanduser('../back/ml-100k/u.data')
    reader = Reader(line_format='user item rating timestamp', sep='\t')
    data = Dataset.load_from_file(file_path, reader=reader)

    #Treinando o modelo
    trainset = data.build_full_trainset()

    model = KNNBasic()
    model.fit(trainset)
    return model

Com isso, __pode-se treinar um modelo__, por exemplo, para um usuário que gostou de Toy Story (1995), GoldenEye (1995), Four Rooms (1995), Get Shorty (1995) e Copycat (1995) (respectivamente __IDs 1,2,3,4 e 5__).

In [34]:
watched_movies = [1,2,3,4,5]
model = train(watched_movies)

Computing the msd similarity matrix...
Done computing similarity matrix.


### 2.3 Previsões

Uma vez com um modelo definido, foi feita uma __previsão de nota para cada filme__ na tabela para o usuário que foi criado, essas notas foram então ordenadas em ordem decrescente:

In [40]:
import operator
def get_all_ratings(user, model, movie_list):
    """
    Pega as notas previstas para cada um dos filmes da tabela de filmes
    :param user: Usuário que deseja-se obter as previsões 
    :param algo: Modelo treinado
    :param movie_list: Lista de filmes do usuário
    :return: Retorna as notas previstas
    """
    uid = str(user)
    preds = []
    for movie in movies:
        #Descarta-se os filmes que já estavam na lista, pois não é desejado recomendar o mesmo filme
        if movie["movie_id"] in movie_list:
            continue

        iid = str(movie["movie_id"])
        pred = model.predict(uid, iid)
        preds.append({"movie_id": iid, "rating": pred.est})

    preds.sort(key=operator.itemgetter('rating'), reverse=True)
    return preds

Agora é possível obter as previsões para o usuário do modelo que foi treinado:

In [43]:
preds = get_all_ratings(DEFAULT_USER, model, watched_movies)

Estes são os vinte filmes mais indicados para este usuário:

In [44]:
df = pd.DataFrame(data=preds[:20])
cols = ["movie_id", "rating"]
df = df[cols]
df

,movie_id,rating
0,1,5.000000
1,814,5.000000
2,1189,5.000000
3,1463,5.000000
4,1467,5.000000
5,1472,5.000000
6,1500,5.000000
7,1536,5.000000
8,1599,5.000000
9,1656,5.000000


### 2.4 Melhorando a tabela resultado

Como o usuário não foi apresentado com o conceito de notas, será utilizada uma métrica de __relevância__, essa métrica será calculada como uma __razão entre a nota apresentada e a nota máxima__, e será expressa em __porcentagem__. Além disso, também é desejável obter expor ao usuário os nomes dos filmes, ao invés de ids. Para isso, será utilizada a __tabela de itens__ e será feito o __cruzamento com os ids__ obtidos.

In [55]:
movie_dict = {}
for movie in movies:
    movie_dict[movie["movie_id"]] = movie["name"]

output_dataframe =[]
for pred in preds:
    output_row ={}
    output_row["Nome do Filme"] = movie_dict[pred["movie_id"]] 
    relevance = (pred["rating"] / float(MAX_RATING)) * 100
    output_row["Relevância"] = "%.1f %%" %relevance
    output_dataframe.append(output_row)

df = pd.DataFrame(data=output_dataframe[:20])
cols = ["Nome do Filme", "Relevância"]
df = df[cols]
df

,Nome do Filme,Relevância
0,Toy Story (1995),100.0 %
1,"Great Day in Harlem, A (1994)",100.0 %
2,Prefontaine (1997),100.0 %
3,"Boys, Les (1997)",100.0 %
4,"Saint of Fort Washington, The (1993)",100.0 %
5,"Visitors, The (Visiteurs, Les) (1993)",100.0 %
6,Santa with Muscles (1996),100.0 %
7,Aiqing wansui (1994),100.0 %
8,Someone Else's America (1995),100.0 %
9,Little City (1998),100.0 %


## Resultados

Uma __página com o resultado__ do projeto aqui documentado, pode ser encontrada em __[http://104.196.53.19/](http://104.196.53.19/)__.    


__OBS__.: A __proposta inicial__ do projeto era de que se pegasse um usuário existente do banco e buscasse indicações para esse usuário. Conversei com o professor e achei que seria mais útil e interessante fazer inserções e criar um novo usuário, para que uma recomendação real fosse feita, o professor aceitou a sugestão.